# Microburbs
## Mini Project


Author: Midhun Shyam

Question 2:
Orientation: Find out for every house which direction it faces (eg is it north facing?). Produce an output file where it's one row per property and include columns for address and orientation.



### Notebook setup (paths, toggles)

In [1]:
# JUPYTER CELL 0 — CONFIG
from pathlib import Path

# Input files
GNAF_PARQUET = Path("gnaf_prop.parquet")
ROADS_GPKG   = Path("roads.gpkg")

# Outputs
RESULTS_CSV  = Path("gnaf_orientation.csv")
OUT_QC_CSV   = Path("gnaf_orientation_validated.csv")

# Parameters
EPSG_WGS84   = 4326
EPSG_METRIC  = 7856  # GDA2020 / MGA Zone 56 (Sydney/NSW)
RADIUS_LIST  = [20, 40, 80, 160, 320, 500]  # metres for candidate road search

SHOW_SAMPLES = 5  # how many rows to preview


### Imports + display helpers

In [2]:
# JUPYTER CELL 1 — IMPORTS
import math
import numpy as np
import pandas as pd
import geopandas as gpd
import pyarrow.parquet as pq
from shapely.geometry import Point, box
from tqdm import tqdm

pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 140)

def h1(title: str):
    print("\n" + "="*len(title))
    print(title)
    print("="*len(title))


### Geometry helpers

In [3]:
# JUPYTER CELL 2 — HELPERS

def bearing_deg(p_from, p_to):
    """Bearing (degrees) from p_from -> p_to in EPSG:4326 (lat, lon)."""
    lat1, lon1 = math.radians(p_from.y), math.radians(p_from.x)
    lat2, lon2 = math.radians(p_to.y), math.radians(p_to.x)
    dlon = lon2 - lon1
    x = math.sin(dlon) * math.cos(lat2)
    y = math.cos(lat1)*math.sin(lat2) - math.sin(lat1)*math.cos(lat2)*math.cos(dlon)
    brng = math.degrees(math.atan2(x, y))
    return (brng + 360) % 360

def bucket_compass8(deg):
    """Map degrees to 8-wind compass."""
    dirs = ["N","NE","E","SE","S","SW","W","NW"]
    step = 360 / 8
    return dirs[int((deg + step/2) // step) % 8]

def nearest_point_on_line(pt, line):
    """Nearest point on a (multi)line to a point (same CRS)."""
    # Handle MultiLineString by projecting to each component
    if line.geom_type == "MultiLineString":
        best_d = 1e18
        best_pt = None
        for seg in line.geoms:
            cand = seg.interpolate(seg.project(pt))
            d = pt.distance(cand)
            if d < best_d:
                best_d, best_pt = d, cand
        return best_pt
    return line.interpolate(line.project(pt))


### Load GNAF (Parquet) and preview

In [4]:
# JUPYTER CELL 3 — LOAD GNAF
h1("Loading GNAF (Parquet)")

table = pq.read_table(GNAF_PARQUET, use_pandas_metadata=False)
print("Schema:\n", table.schema)
print(f"Rows: {table.num_rows:,} | Columns: {table.num_columns}")

df_gnaf = table.to_pandas()
print("\nSample:")
display(df_gnaf.head(SHOW_SAMPLES))

# Make GeoDataFrame (requires longitude/latitude)
if {"longitude", "latitude"}.issubset(df_gnaf.columns):
    gnaf_gdf = gpd.GeoDataFrame(
        df_gnaf.copy(),
        geometry=gpd.points_from_xy(df_gnaf["longitude"], df_gnaf["latitude"]),
        crs=f"EPSG:{EPSG_WGS84}",
    )
    print("\nGNAF GeoDataFrame info:")
    display(gnaf_gdf.info())
    print("Bounding box:", gnaf_gdf.total_bounds)
else:
    raise ValueError("No 'longitude'/'latitude' columns found in GNAF.")



Loading GNAF (Parquet)
Schema:
 gid: int64
gnaf_pid: string
street_locality_pid: string
locality_pid: string
alias_principal: string
primary_secondary: string
building_name: string
lot_number: string
flat_number: string
level_number: string
number_first: string
number_last: string
street_name: string
street_type: string
street_suffix: string
address: string
locality_name: string
postcode: string
state: string
locality_postcode: string
confidence: int64
legal_parcel_id: string
mb_2016_code: int64
mb_2021_code: int64
latitude: double
longitude: double
geocode_type: string
reliability: int64
geom: string
schema: string
__index_level_0__: int64
-- schema metadata --
pandas: '{"index_columns": ["__index_level_0__"], "column_indexes": [{"na' + 3970
Rows: 70,591 | Columns: 31

Sample:


,gid,gnaf_pid,street_locality_pid,locality_pid,alias_principal,primary_secondary,building_name,lot_number,flat_number,level_number,number_first,number_last,street_name,street_type,street_suffix,address,locality_name,postcode,state,locality_postcode,confidence,legal_parcel_id,mb_2016_code,mb_2021_code,latitude,longitude,geocode_type,reliability,geom,schema
0,3115,GANSW705858629,NSW2878308,locc9618e9a6979,P,None,None,1,None,None,10,None,MELNOTTE,AVENUE,None,10 MELNOTTE AVENUE,ROSEVILLE,2069,NSW,2069,2,1/105203,10840311000,10840311000,-33.787478,151.182595,PROPERTY CENTROID,2,0101000020BB100000E599B8D1D7E562406816D312CCE4...,gnaf_202402
1,3120,GANSW705858630,NSW2878308,locc9618e9a6979,P,None,None,26,None,None,11,None,MELNOTTE,AVENUE,None,11 MELNOTTE AVENUE,ROSEVILLE,2069,NSW,2069,2,26/7045,10840312000,10840312000,-33.787164,151.183209,PROPERTY CENTROID,2,0101000020BB1000008215D0D9DCE562403E3335C7C1E4...,gnaf_202402
2,3123,GANSW705858631,NSW2878308,locc9618e9a6979,P,None,None,16,None,None,12,None,MELNOTTE,AVENUE,None,12 MELNOTTE AVENUE,ROSEVILLE,2069,NSW,2069,2,15/455712,10840311000,10840311000,-33.787375,151.182549,PROPERTY CENTROID,2,0101000020BB100000E8F3B470D7E562406A31BAB4C8E4...,gnaf_202402
3,3126,GANSW705858632,NSW2878308,locc9618e9a6979,P,None,None,27,None,None,13,None,MELNOTTE,AVENUE,None,13 MELNOTTE AVENUE,ROSEVILLE,2069,NSW,2069,2,27/7045,10840312000,10840312000,-33.787023,151.183146,PROPERTY CENTROID,2,0101000020BB1000009DEE2354DCE56240D5DAEA28BDE4...,gnaf_202402
4,3130,GANSW705858633,NSW2878308,locc9618e9a6979,P,None,None,1,None,None,14,None,MELNOTTE,AVENUE,None,14 MELNOTTE AVENUE,ROSEVILLE,2069,NSW,2069,2,1/963791,10840311000,10840311000,-33.787273,151.182502,PROPERTY CENTROID,2,0101000020BB1000005048190ED7E5624024456458C5E4...,gnaf_202402



GNAF GeoDataFrame info:
<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 70591 entries, 0 to 17674
Data columns (total 31 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   gid                  70591 non-null  int64   
 1   gnaf_pid             70591 non-null  object  
 2   street_locality_pid  70591 non-null  object  
 3   locality_pid         70591 non-null  object  
 4   alias_principal      70591 non-null  object  
 5   primary_secondary    23062 non-null  object  
 6   building_name        1005 non-null   object  
 7   lot_number           50669 non-null  object  
 8   flat_number          20738 non-null  object  
 9   level_number         1006 non-null   object  
 10  number_first         70584 non-null  object  
 11  number_last          5132 non-null   object  
 12  street_name          70591 non-null  object  
 13  street_type          68424 non-null  object  
 14  street_suffix        4 non-null      objec

None

Bounding box: [151.16174945 -33.8162863  151.23148752 -33.76884754]


### Load Roads (GPKG), align CRS, preview

In [5]:
# JUPYTER CELL 4 — LOAD ROADS
h1("Loading Roads (GeoPackage)")

roads = gpd.read_file(ROADS_GPKG)
if roads.crs is None:
    roads = roads.set_crs(EPSG_WGS84)
else:
    roads = roads.to_crs(EPSG_WGS84)

print("Roads info:")
display(roads.info())
print("Geometry types:", roads.geom_type.unique())
print("CRS:", roads.crs)
print("Bounding box:", roads.total_bounds)
print("\nAttribute columns:", list(roads.columns))
display(roads.head(SHOW_SAMPLES))



Loading Roads (GeoPackage)
Roads info:
<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 173 entries, 0 to 172
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   osm_id      173 non-null    object  
 1   code        173 non-null    int64   
 2   fclass      173 non-null    object  
 3   name        73 non-null     object  
 4   layer       173 non-null    float64 
 5   bridge      173 non-null    object  
 6   tunnel      173 non-null    object  
 7   fname       173 non-null    object  
 8   type        0 non-null      object  
 9   width       0 non-null      object  
 10  population  0 non-null      object  
 11  ref         0 non-null      object  
 12  oneway      173 non-null    object  
 13  maxspeed    173 non-null    float64 
 14  geometry    173 non-null    geometry
dtypes: float64(2), geometry(1), int64(1), object(11)
memory usage: 20.4+ KB


None

Geometry types: ['LineString']
CRS: EPSG:4326
Bounding box: [151.2084668 -33.806042  151.230767  -33.794718 ]

Attribute columns: ['osm_id', 'code', 'fclass', 'name', 'layer', 'bridge', 'tunnel', 'fname', 'type', 'width', 'population', 'ref', 'oneway', 'maxspeed', 'geometry']


,osm_id,code,fclass,name,layer,bridge,tunnel,fname,type,width,population,ref,oneway,maxspeed,geometry
0,1081034612,5154,path,None,1.0,T,F,gis_osm_roads_free_1,None,None,None,None,B,0.0,"LINESTRING (151.22193 -33.80488, 151.22195 -33..."
1,868422498,5155,steps,None,0.0,F,F,gis_osm_roads_free_1,None,None,None,None,B,0.0,"LINESTRING (151.22117 -33.80514, 151.22118 -33..."
2,821720812,5154,path,None,0.0,F,F,gis_osm_roads_free_1,None,None,None,None,B,0.0,"LINESTRING (151.21766 -33.80537, 151.21793 -33..."
3,507890155,5141,service,None,0.0,F,F,gis_osm_roads_free_1,None,None,None,None,B,0.0,"LINESTRING (151.21155 -33.79623, 151.21155 -33..."
4,1081035341,5154,path,Harold Reid Foreshore Track,0.0,F,F,gis_osm_roads_free_1,None,None,None,None,B,0.0,"LINESTRING (151.21201 -33.79623, 151.2121 -33...."


### Quick spatial overlap sanity check

In [6]:
# JUPYTER CELL 5 — OVERLAP CHECK
h1("Spatial Overlap Check")

bbox_gnaf = box(*gnaf_gdf.total_bounds)
bbox_roads = box(*roads.total_bounds)
overlap = bbox_gnaf.intersects(bbox_roads)
print(f"Do the datasets overlap spatially? {overlap}")

# Clip GNAF to roads bbox to reduce workload
if overlap:
    minx, miny, maxx, maxy = roads.total_bounds
    gnaf_near = gnaf_gdf.cx[minx:maxx, miny:maxy].copy()
else:
    gnaf_near = gnaf_gdf.copy()

print(f"Using {len(gnaf_near):,} nearby properties (within roads bbox).")



Spatial Overlap Check
Do the datasets overlap spatially? True
Using 6,728 nearby properties (within roads bbox).


 ### Compute nearest road + orientation (write

In [7]:
# JUPYTER CELL 6 — ORIENTATION COMPUTE
h1("Compute nearest road + orientation")

# Project to metric CRS for distances/buffers
gnaf_m  = gnaf_near.to_crs(EPSG_METRIC)
roads_m = roads.to_crs(EPSG_METRIC)

# Spatial index (works for rtree or pygeos/shapely STRtree backends)
sidx = roads_m.sindex

results = []
for i, row in tqdm(gnaf_m.iterrows(), total=len(gnaf_m), desc="Properties"):
    pt = row.geometry

    # Progressive search radius to find candidate roads
    idxs = None
    for r in RADIUS_LIST:
        idxs = list(sidx.query(pt.buffer(r)))
        if idxs:
            break

    if not idxs:
        results.append({
            "address": row.get("address"),
            "orientation": None,
            "bearing_deg": np.nan,
            "distance_to_road_m": np.nan,
            "note": "no_road_within_max_radius",
        })
        continue

    # Choose true nearest by projected distance
    best_d = 1e18
    best_np = None
    for ridx in idxs:
        line = roads_m.geometry.iloc[ridx]
        np_on = nearest_point_on_line(pt, line)
        d = pt.distance(np_on)
        if d < best_d:
            best_d, best_np = d, np_on

    # Convert to WGS84 for bearing
    pt_wgs = gpd.GeoSeries([pt], crs=EPSG_METRIC).to_crs(EPSG_WGS84).iloc[0]
    np_wgs = gpd.GeoSeries([best_np], crs=EPSG_METRIC).to_crs(EPSG_WGS84).iloc[0]
    brg = bearing_deg(pt_wgs, np_wgs)

    results.append({
        "address": row.get("address"),
        "orientation": bucket_compass8(brg),
        "bearing_deg": round(float(brg), 1),
        "distance_to_road_m": round(float(best_d), 2),
        "note": "ok",
    })

out = pd.DataFrame(results)
out.to_csv(RESULTS_CSV, index=False)
print(f"✔ Saved {len(out):,} rows to {RESULTS_CSV}")
display(out.head(SHOW_SAMPLES))



Compute nearest road + orientation


Properties: 100%|██████████| 6728/6728 [00:04<00:00, 1632.72it/s]


✔ Saved 6,728 rows to gnaf_orientation.csv


,address,orientation,bearing_deg,distance_to_road_m,note
0,7 CHARLES STREET,N,6.7,14.52,ok
1,22 THE PARAPET,SE,155.5,16.52,ok
2,24 THE PARAPET,S,177.5,20.84,ok
3,27 THE PARAPET,SW,220.3,28.35,ok
4,109 EASTERN VALLEY WAY,E,70.0,58.34,ok


### Validation pass (recompute + compare to CSV)

In [8]:
# JUPYTER CELL 7 — VALIDATION
h1("Validation: recompute and compare to CSV")

# Minimal GNAF columns for join
keep_cols = ["address", "latitude", "longitude", "street_name", "street_type", "street_suffix"]
src = gnaf_gdf.copy()
for c in keep_cols:
    if c not in src.columns:
        src[c] = None
src = src[keep_cols].copy()
src["address_norm"] = src["address"].astype(str).str.strip()

# Load computed results
res = pd.read_csv(RESULTS_CSV)
res["address_norm"] = res["address"].astype(str).str.strip()
res["row_id"] = np.arange(len(res))

# Join by normalized address
merged = res.merge(src, on="address_norm", how="left", suffixes=("", "_gnaf"), indicator=True)

# Count matches per result row
match_counts = merged.groupby("row_id")["_merge"].count().rename("n_matches")
merged = merged.merge(match_counts, on="row_id", how="left")

# Keep first match per row_id
merged = merged.sort_values("row_id").drop_duplicates(subset=["row_id"], keep="first").reset_index(drop=True)

# Build GeoDataFrame from joined lon/lat
props = gpd.GeoDataFrame(
    merged,
    geometry=gpd.points_from_xy(merged["longitude"], merged["latitude"]),
    crs=EPSG_WGS84,
)

# Clip to roads bbox (safety)
minx, miny, maxx, maxy = roads.total_bounds
props = props.cx[minx:maxx, miny:maxy].copy()

print(f"Validating {len(props):,} rows (of {len(res):,}). Unmatched/out-of-bounds reported as missing.")



Validation: recompute and compare to CSV
Validating 6,728 rows (of 6,728). Unmatched/out-of-bounds reported as missing.


In [9]:
# JUPYTER CELL 8 — VALIDATION GEOMETRY + SINDEX
props_m  = props.to_crs(EPSG_METRIC)
roads_m  = roads.to_crs(EPSG_METRIC)
sidx     = roads_m.sindex

re_bearing = np.full(len(props_m), np.nan, dtype=float)
re_dist    = np.full(len(props_m), np.nan, dtype=float)
re_orient  = np.array([None]*len(props_m), dtype=object)
re_note    = np.array([""]*len(props_m), dtype=object)

for i, row in tqdm(props_m.iterrows(), total=len(props_m), desc="Recomputing"):
    pt = row.geometry
    idxs = None
    for r in RADIUS_LIST:
        idxs = list(sidx.query(pt.buffer(r)))
        if idxs:
            break
    if not idxs:
        re_note[i] = "no_road_within_max_radius"
        continue

    # Nearest road geometry
    best_d = 1e18
    best_np = None
    for ridx in idxs:
        line = roads_m.geometry.iloc[ridx]
        np_on = nearest_point_on_line(pt, line)
        d = pt.distance(np_on)
        if d < best_d:
            best_d, best_np = d, np_on

    # Bearing in WGS84
    pt_wgs = gpd.GeoSeries([pt], crs=EPSG_METRIC).to_crs(EPSG_WGS84).iloc[0]
    np_wgs = gpd.GeoSeries([best_np], crs=EPSG_METRIC).to_crs(EPSG_WGS84).iloc[0]
    brg = bearing_deg(pt_wgs, np_wgs)
    re_bearing[i] = brg
    re_dist[i]    = best_d
    re_orient[i]  = bucket_compass8(brg)
    re_note[i]    = "ok"

props_m["re_bearing_deg"]      = np.round(re_bearing, 1)
props_m["re_distance_to_road"] = np.round(re_dist, 2)
props_m["re_orientation"]      = re_orient
props_m["re_note"]             = re_note


Recomputing: 100%|██████████| 6728/6728 [00:04<00:00, 1628.33it/s]


In [10]:
# JUPYTER CELL 9 — COMPARISON + QC VERDICTS
# Rename original CSV columns for clarity
props_m = props_m.rename(columns={
    "orientation": "csv_orientation",
    "bearing_deg": "csv_bearing_deg",
    "distance_to_road_m": "csv_distance_to_road_m",
})

# Booleans + diffs
props_m["orientation_match"] = (props_m["csv_orientation"].astype(str) == props_m["re_orientation"].astype(str))
props_m["bearing_diff_deg"]  = np.abs(props_m["csv_bearing_deg"] - props_m["re_bearing_deg"])
props_m["distance_diff_m"]   = np.abs(props_m["csv_distance_to_road_m"] - props_m["re_distance_to_road"])

# Join flags
props_m["ambiguous_address"] = props_m["n_matches"].fillna(0).astype(int).gt(1)
props_m["unmatched_address"] = props_m["_merge"].ne("both")

def qc_verdict(row):
    if row["unmatched_address"]:
        return "unmatched_address"
    if row["re_note"] == "no_road_within_max_radius":
        return "no_road_nearby"
    if not row["orientation_match"]:
        return "orientation_mismatch"
    return "ok"

props_m["qc_verdict"] = props_m.apply(qc_verdict, axis=1)

cols_out = [
    "address",
    "csv_orientation", "csv_bearing_deg", "csv_distance_to_road_m",
    "re_orientation", "re_bearing_deg", "re_distance_to_road",
    "orientation_match", "bearing_diff_deg", "distance_diff_m",
    "ambiguous_address", "unmatched_address", "qc_verdict"
]
props_m[cols_out].to_csv(OUT_QC_CSV, index=False)

total = len(props_m)
acc   = int(props_m["orientation_match"].sum())
print(f"✔ Wrote {OUT_QC_CSV}")
print(f"Overall orientation match: {acc}/{total} = {acc/total:.1%}")
print("\nQC verdict counts:")
display(props_m["qc_verdict"].value_counts(dropna=False))
print("\nExamples of mismatches:")
display(props_m.loc[props_m["qc_verdict"]=="orientation_mismatch", cols_out].head(10))


✔ Wrote gnaf_orientation_validated.csv
Overall orientation match: 6728/6728 = 100.0%

QC verdict counts:


qc_verdict
ok    6728
Name: count, dtype: int64


Examples of mismatches:


,address,csv_orientation,csv_bearing_deg,csv_distance_to_road_m,re_orientation,re_bearing_deg,re_distance_to_road,orientation_match,bearing_diff_deg,distance_diff_m,ambiguous_address,unmatched_address,qc_verdict


### Optional: Investigate mismatches quickly

In [11]:
# JUPYTER CELL 10 — QUICK DIAGNOSTICS (OPTIONAL)

# Where bearing differs a lot (e.g., > 90°) and distance is close
big_angle = props_m.query("qc_verdict == 'orientation_mismatch' and bearing_diff_deg >= 90")
display(big_angle.head(10))

# Where the validation found a much closer road than the original (~ > 30m diff)
closer_road = props_m.query("qc_verdict == 'orientation_mismatch' and distance_diff_m >= 30")
display(closer_road.head(10))


,address,csv_orientation,csv_bearing_deg,csv_distance_to_road_m,note,address_norm,row_id,address_gnaf,latitude,longitude,street_name,street_type,street_suffix,_merge,n_matches,geometry,re_bearing_deg,re_distance_to_road,re_orientation,re_note,orientation_match,bearing_diff_deg,distance_diff_m,ambiguous_address,unmatched_address,qc_verdict


,address,csv_orientation,csv_bearing_deg,csv_distance_to_road_m,note,address_norm,row_id,address_gnaf,latitude,longitude,street_name,street_type,street_suffix,_merge,n_matches,geometry,re_bearing_deg,re_distance_to_road,re_orientation,re_note,orientation_match,bearing_diff_deg,distance_diff_m,ambiguous_address,unmatched_address,qc_verdict
